In [1]:
import os
import sys


data_dir = os.path.join('.', 'data')

print(sys.version)

3.5.5 |Anaconda, Inc.| (default, May 13 2018, 21:12:35) 
[GCC 7.2.0]


In [2]:
from lib.data import download_data

download_data(data_dir)

In [3]:
from lib.load_dataset import load_datasets

df = load_datasets(data_dir, 2015, 2017)

print(df.shape)
df.head()

Loading datasets from year 2015 to 2017


/opt/conda/lib/python3.5/site-packages/pandas/core/frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


(388072, 25)


,antall_stillinger,arbeidssted_fylke,arbeidssted_fylkesnummer,arbeidssted_kommune,arbeidssted_kommunenummer,arbeidssted_land,arbeidssted_landkode,isco_versjon,nav_enhet_kode,offisiell_statistikk_flagg,...,stilling_kilde,stillingsbeskrivelse,stillingsnummer,stillingstittel,virksomhet_navn,virksomhet_organisasjonsnr,yrke,yrke_grovgruppe,yrkesbetegnelse,yrkeskode
0,1,Østfold,01,Halden,0101,Norge,NO,ISCO-08,101,1,...,Annonsert i media,# senter for framandspråk i opplæringa er det ...,101201412000010,Barnehagelærar / Førstekonsulent (14/04241),HØGSKOLEN I ØSTFOLD,973967592,Førskolelærere,Undervisning,Førskolelærer,2342
1,1,Østfold,01,Halden,0101,Norge,NO,ISCO-08,101,1,...,Overført fra arbeidsgiver,Jobbnorge ID: 108617<br><br><p dir=#LTR# align...,101201412000011,Halden fengsel - ledig fast stilling som verks...,HALDEN FENGSEL,993315060,Fengselsbetjenter,Serviceyrker og annet arbeid,Fengselsbetjent,5413
2,1,Østfold,01,Halden,0101,Norge,NO,ISCO-08,101,1,...,Overført fra arbeidsgiver,Jobbnorge ID: 108301<br><br><p dir=#LTR# align...,101201412000012,Halden fengsel - ledig fast stilling som verks...,HALDEN FENGSEL,993315060,Fengselsbetjenter,Serviceyrker og annet arbeid,Fengselsbetjent,5413
3,1,Østfold,01,Halden,0101,Norge,NO,ISCO-08,101,1,...,Annonsert i media,# har ambisjoner om å vokse og styrker teamet ...,101201412000013,Key Account Manager,SMART INNOVATION NORWAY AS,886332882,Selgere (engros),Butikk- og salgsarbeid,Key Account Manager (salg),3322
4,1,Østfold,01,Halden,0101,Norge,NO,ISCO-08,101,1,...,Annonsert i media,# is now expanding its leading team with senio...,101201412000014,Senior Reasearchers/Reasearchers,SMART INNOVATION NORWAY AS,886332882,Sivilingeniører (elektronikk),Ingeniør- og ikt-fag,Forsker (elektronikk),2152


In [4]:
import re
from lib.preprocessing import remove_tags, unescape_html, remove_whitespace

non_alphanumeric_pattern = re.compile(r'\W')

# Select subset of columns
col = [
    'stillingsbeskrivelse', 
    'yrke_grovgruppe',
    'stilling_kilde',
]
df = df[col]


# Preprocess text and add some columns
df['stillingsbeskrivelse'] = df['stillingsbeskrivelse']\
    .map(remove_tags)\
    .map(unescape_html)\
    .map(remove_whitespace)\
    .map(lambda x: non_alphanumeric_pattern.sub(' ', x))\
    #.map(lambda x: x.lower())
df['is_from_nav'] = df['stilling_kilde'].map(lambda x: x=='Reg av arb.giver på nav.no')
df['from_media'] = df['stilling_kilde'].map(lambda x: x=='Annonsert i media')
df['is_healthcare'] = df['yrke_grovgruppe'].map(lambda x: x=='Helse, pleie og omsorg')
df['is_industrial'] = df['yrke_grovgruppe'].map(lambda x: x=='Industriarbeid')

print(df.shape)
df.head()




(388072, 7)


,stillingsbeskrivelse,yrke_grovgruppe,stilling_kilde,is_from_nav,from_media,is_healthcare,is_industrial
0,senter for framandspråk i opplæringa er det ...,Undervisning,Annonsert i media,False,True,False,False
1,Jobbnorge ID 108617 fengsel er det ledig fas...,Serviceyrker og annet arbeid,Overført fra arbeidsgiver,False,False,False,False
2,Jobbnorge ID 108301 fengsel er det ledig fas...,Serviceyrker og annet arbeid,Overført fra arbeidsgiver,False,False,False,False
3,har ambisjoner om å vokse og styrker teamet ...,Butikk- og salgsarbeid,Annonsert i media,False,True,False,False
4,is now expanding its leading team with senio...,Ingeniør- og ikt-fag,Annonsert i media,False,True,False,False


In [5]:
import csv
from sklearn.model_selection import train_test_split

label_column = 'yrke_grovgruppe'

def make_dataframe_fasttext_format(df, label_col, size=None, random_seed=305):
    if not size:
        size = len(df)
    result = df.sample(n=size, random_state=random_seed)[[label_col, 'stillingsbeskrivelse']]
    result[label_col] = result[label_col].map(lambda x: str(x).replace(' ', '_'))
    result[label_col] = result[label_col].map(lambda x: '__label__' + str(x))
    return result


out = make_dataframe_fasttext_format(df, label_column)

def write_train_test_files_for_fasttext(df, name, test_frac=0.2):
    train_df, test_df = train_test_split(df, test_size=test_frac)
    test_df.to_csv(os.path.join('.', 'fasttext', 'data', ''.join([name, '.test'])), escapechar=' ', sep=' ', header=None, mode='w', index=False, quoting=csv.QUOTE_NONE)
    train_df.to_csv(os.path.join('.', 'fasttext', 'data', ''.join([name, '.train'])), escapechar=' ', sep=' ', header=None, mode='w', index=False, quoting=csv.QUOTE_NONE)
    

write_train_test_files_for_fasttext(out, label_column)